In [25]:
from hakai_api import Client
import mercury as mr
import pandas as pd 
import plotly.express as px
import folium
from folium.plugins import FastMarkerCluster

In [31]:

def get_hakai_ctd_data(crendentials, station, start_time,end_time, fields=['depth','start_dt']):
    client = Client(credentials=crendentials or None)
    url = ''.join(
        f"{client.api_root}/ctd/views/file/cast/data?"
        + f"station={station}"
        + f"&start_dt>{start_time}"
        + f"&start_dt<{end_time}"
        + "&direction_flag=d"
        + f"&fields={','.join(fields)}"
        + "&limit=-1"
    )
    response = client.get(url)
    return pd.DataFrame(response.json())

In [32]:
app = mr.App(title="Hakai CTD", description="Generate  CTD plot") 

mercury.App

In [38]:
credentials = mr.Text(label="Get hakai credentials from: https://hecate.hakai.org/api-client-login ")
station = mr.Text(value="QU39", 
                   label="Select station")
start_time = mr.Text(value="2022-01-01",label='Start Date (yyyy-mm-dd)')
end_time = mr.Text(value="2023-01-01",label='End Date   (yyyy-mm-dd)')

mercury.Text

mercury.Text

mercury.Text

mercury.Text

# Hakai CTD Data

## Hakai Credentials
Retrieve the hakai credentials from: https://hecate.hakai.org/api-client-login


In [39]:
fields= ['start_dt','depth','temperature','hakai_id','latitude','longitude']
if not credentials.value:
    df = pd.DataFrame({field: [] for field in fields})
else:
    df = get_hakai_ctd_data(credentials.value,station.value,start_time.value,end_time.value,fields=fields)


## Map

In [47]:
if not df.empty:
    m = folium.Map(location=df[['latitude','longitude']].mean())
    FastMarkerCluster(data=df.groupby('hakai_id').first()[['latitude','longitude']].dropna(axis=0).values).add_to(m)
else:
    m=None
m


## Time-Series Profiles

In [48]:
if not df.empty:
    fig = px.scatter(df,x='start_dt',y='depth',color='temperature',hover_name='hakai_id')
    fig.update_yaxes(autorange='reversed')
else:
    fig = None
fig
